In [4]:
import pandas as pd

In [5]:
# === Step 1: Load and prepare the option data ===

# Load the option data CSV
option_data = pd.read_csv("Data/qqq_option_data_2018_2023.csv")

# Convert date columns to datetime
option_data['date'] = pd.to_datetime(option_data['date'])
option_data['exdate'] = pd.to_datetime(option_data['exdate'])

# Prompt the user for the analysis date
user_input = input("Enter an analysis date (YYYY-MM-DD): ")
analysis_date = pd.to_datetime(user_input)

# Check if the date exists in the dataset
available_dates = option_data['date'].unique()
if analysis_date not in available_dates:
    raise ValueError(f"Date {analysis_date.date()} not found in dataset. Available dates range from {min(available_dates).date()} to {max(available_dates).date()}.")

# Filter the data for that specific date
filtered_data = option_data[option_data['date'] == analysis_date].copy()

# Drop any rows with missing delta or gamma
filtered_data = filtered_data.dropna(subset=['delta', 'gamma'])

# Reset index
filtered_data.reset_index(drop=True, inplace=True)

# Preview the cleaned data
print(f"\nOptions data for {analysis_date.date()}:")
print(filtered_data[['exdate', 'cp_flag', 'strike_price', 'delta', 'gamma']].head())



Options data for 2018-12-31:
      exdate cp_flag  strike_price     delta     gamma
0 2019-01-25       P      156000.0 -0.562582  0.032647
1 2019-01-25       P      156500.0 -0.579636  0.032672
2 2019-01-25       P      157000.0 -0.596752  0.032623
3 2019-01-25       P      157500.0 -0.613719  0.032466
4 2019-01-25       P      158000.0 -0.630612  0.032233


In [7]:
# === Step 2: Define your portfolio ===

# Set your simulated portfolio value and volatility target
portfolio_value = 10_000_000   # $10 million in QQQ
vol_target = 0.20              # 20% annual volatility target

# Fix strike price scale (if needed)
# QQQ strike prices may be stored as cents or multiplied by 1000
if filtered_data['strike_price'].max() > 1000:
    filtered_data['strike_price'] = filtered_data['strike_price'] / 100

# Filter for near-the-money (ATM) options to estimate QQQ price
# Estimate QQQ price from option data (midpoint of strike prices near-the-money)
# We'll use the average strike of ATM options for simplicity
# ATM options typically have delta between ~0.4 and ~0.6
# You can customize this based on how close to the market price you want
atm_options = filtered_data[(filtered_data['delta'] > 0.4) & (filtered_data['delta'] < 0.6)]

# Handle case where no ATM options are found
if atm_options.empty:
    raise ValueError("No ATM options found for this date. Try another date or adjust delta range.")

# Estimate QQQ price using the average strike price of ATM options
estimated_qqq_price = atm_options['strike_price'].mean()

# Calculate number of QQQ shares in the portfolio
num_shares = portfolio_value / estimated_qqq_price

# Assume each QQQ share has delta ≈ +1
portfolio_delta = num_shares * 1  # Adjust if using actual QQQ delta

# Display the results
print(f"\nPortfolio defined for {analysis_date.date()}:")
print(f"- Portfolio value: ${portfolio_value:,.2f}")
print(f"- Estimated QQQ price: ${estimated_qqq_price:,.2f}")
print(f"- Number of shares: {num_shares:,.2f}")
print(f"- Portfolio delta: {portfolio_delta:,.2f}")



Portfolio defined for 2018-12-31:
- Portfolio value: $10,000,000.00
- Estimated QQQ price: $1,543.91
- Number of shares: 6,477.08
- Portfolio delta: 6,477.08


In [8]:
# === Step 3: Select Hedging Options ===

# 1. For Delta Hedging → pick PUT options with negative delta
put_options = filtered_data[(filtered_data['cp_flag'] == 'P') & (filtered_data['delta'] < 0)]

# 2. For Gamma Hedging → pick ATM calls or puts with high gamma (usually near 0.4 < delta < 0.6)
high_gamma_options = filtered_data[(filtered_data['gamma'] > filtered_data['gamma'].quantile(0.90))]

# Optional: sort by gamma to pick the highest one
high_gamma_options = high_gamma_options.sort_values(by='gamma', ascending=False)

# Preview selections
print(f"\nTop Put Options for Delta Hedging on {analysis_date.date()}:")
print(put_options[['exdate', 'strike_price', 'delta', 'gamma']].head())

print(f"\nTop High-Gamma Options for Gamma Hedging on {analysis_date.date()}:")
print(high_gamma_options[['cp_flag', 'exdate', 'strike_price', 'delta', 'gamma']].head())



Top Put Options for Delta Hedging on 2018-12-31:
      exdate  strike_price     delta     gamma
0 2019-01-25        1560.0 -0.562582  0.032647
1 2019-01-25        1565.0 -0.579636  0.032672
2 2019-01-25        1570.0 -0.596752  0.032623
3 2019-01-25        1575.0 -0.613719  0.032466
4 2019-01-25        1580.0 -0.630612  0.032233

Top High-Gamma Options for Gamma Hedging on 2018-12-31:
   cp_flag     exdate  strike_price     delta     gamma
79       C 2019-01-25        1565.0  0.417512  0.034303
78       C 2019-01-25        1560.0  0.435435  0.034285
77       C 2019-01-25        1555.0  0.453147  0.034229
80       C 2019-01-25        1570.0  0.399627  0.034205
76       C 2019-01-25        1550.0  0.470649  0.034134


In [10]:
# === Step 4: Calculate Hedge Ratios ===

# Define a function to calculate contracts needed for delta and gamma hedging
def calculate_hedge_ratios(portfolio_delta, portfolio_gamma, option_row):
    """
    Given a selected option row, return delta and gamma hedge contracts needed.
    """
    delta = option_row['delta']
    gamma = option_row['gamma']
    
    delta_hedge = portfolio_delta / abs(delta) if delta != 0 else float('inf')
    gamma_hedge = portfolio_gamma / gamma if gamma != 0 else float('inf')
    
    return delta_hedge, gamma_hedge

# Estimate portfolio gamma (you can adjust this assumption or calculate it from ETF data)
portfolio_gamma = num_shares * 0.002  # assuming ~0.002 gamma per QQQ share

# === Loop through top put options for delta hedging ===
print(" Delta Hedge Options:")
for i in range(min(5, len(put_options))):
    option = put_options.iloc[i]
    delta_contracts, _ = calculate_hedge_ratios(portfolio_delta, portfolio_gamma, option)
    print(f"Put #{i+1}: delta={option['delta']:.4f}, strike={option['strike_price']}, contracts needed={delta_contracts:.2f}")

# === Loop through top high-gamma options for gamma hedging ===
print("Gamma Hedge Options:")
for i in range(min(5, len(high_gamma_options))):
    option = high_gamma_options.iloc[i]
    _, gamma_contracts = calculate_hedge_ratios(portfolio_delta, portfolio_gamma, option)
    print(f"Option #{i+1}: gamma={option['gamma']:.5f}, strike={option['strike_price']}, contracts needed={gamma_contracts:.2f}")


 Delta Hedge Options:
Put #1: delta=-0.5626, strike=1560.0, contracts needed=11513.13
Put #2: delta=-0.5796, strike=1565.0, contracts needed=11174.39
Put #3: delta=-0.5968, strike=1570.0, contracts needed=10853.88
Put #4: delta=-0.6137, strike=1575.0, contracts needed=10553.82
Put #5: delta=-0.6306, strike=1580.0, contracts needed=10271.10
Gamma Hedge Options:
Option #1: gamma=0.03430, strike=1565.0, contracts needed=377.64
Option #2: gamma=0.03429, strike=1560.0, contracts needed=377.84
Option #3: gamma=0.03423, strike=1555.0, contracts needed=378.46
Option #4: gamma=0.03420, strike=1570.0, contracts needed=378.72
Option #5: gamma=0.03413, strike=1550.0, contracts needed=379.51


In [11]:
# === Step 5: Store & Output Hedge Plan ===

# Create a list to hold hedge plan info
hedge_plan = []

# Loop through top 5 delta hedge candidates
for i in range(min(5, len(put_options))):
    option = put_options.iloc[i]
    delta_contracts, _ = calculate_hedge_ratios(portfolio_delta, portfolio_gamma, option)
    hedge_plan.append({
        'type': 'put',
        'strike_price': option['strike_price'],
        'cp_flag': option['cp_flag'],
        'delta': option['delta'],
        'gamma': option['gamma'],
        'contracts_needed': delta_contracts,
        'hedge_type': 'delta'
    })

# Loop through top 5 gamma hedge candidates
for i in range(min(5, len(high_gamma_options))):
    option = high_gamma_options.iloc[i]
    _, gamma_contracts = calculate_hedge_ratios(portfolio_delta, portfolio_gamma, option)
    hedge_plan.append({
        'type': 'call' if option['cp_flag'] == 'C' else 'put',
        'strike_price': option['strike_price'],
        'cp_flag': option['cp_flag'],
        'delta': option['delta'],
        'gamma': option['gamma'],
        'contracts_needed': gamma_contracts,
        'hedge_type': 'gamma'
    })

# Convert to DataFrame
hedge_df = pd.DataFrame(hedge_plan)

# Display it
print("\n Hedge Plan Summary:")
print(hedge_df)

# Optional: export to CSV
# hedge_df.to_csv(f"hedge_plan_{analysis_date.date()}.csv", index=False)



 Hedge Plan Summary:
   type  strike_price cp_flag     delta     gamma  contracts_needed hedge_type
0   put        1560.0       P -0.562582  0.032647      11513.125587      delta
1   put        1565.0       P -0.579636  0.032672      11174.387407      delta
2   put        1570.0       P -0.596752  0.032623      10853.884392      delta
3   put        1575.0       P -0.613719  0.032466      10553.815702      delta
4   put        1580.0       P -0.630612  0.032233      10271.097313      delta
5  call        1565.0       C  0.417512  0.034303        377.639111      gamma
6  call        1560.0       C  0.435435  0.034285        377.837376      gamma
7  call        1555.0       C  0.453147  0.034229        378.455533      gamma
8  call        1570.0       C  0.399627  0.034205        378.721077      gamma
9  call        1550.0       C  0.470649  0.034134        379.508831      gamma
